In [ ]:
# 1. Install Dependencies and Setup

In [ ]:
!pip install tensorflow opencv-python matplotlib

In [ ]:
!pip list

In [ ]:
import tensorflow as tf
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('CPU')
# for gpu in gpus: 
#     tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
# 2. Remove dodgy images

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = 'data' #path for data 

In [ ]:
image_exts = ['jpeg', 'jpg', 'bmp', 'png'] #list of valid img extensions

In [ ]:
for image_class in os.listdir(data_dir): #looping in data for flood or wildfires
    for image in os.listdir(os.path.join(data_dir, image_class)): #looping thru each img in flood or wildfires
        image_path = os.path.join(data_dir, image_class, image)#gets the img in image_path
        try: 
            img = cv2.imread(image_path) #cv2 will save the image in numPy array format
            tip = imghdr.what(image_path)
            if tip not in image_exts:  # check if the img has an extension in 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

In [ ]:
# 3. Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#builds image data set while compiling, automatically reshapes images for consistancy and also batches them up to sie 32
data = tf.keras.utils.image_dataset_from_directory('data')


In [ ]:
data_iterator = data.as_numpy_iterator() #access the data from the image data set or datapipeline

In [ ]:
batch = data_iterator.next() #get another batch frim itr

In [ ]:
# batch[0].shape, represents each image as numpy arrays
#class 0 = Floods
#class 1 = tornadoes
#class 2 = wildfire

#batch[1] = labels

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20)) #using matplotlib to show 4 images 
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# 4. Scale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y)) #data.map allows for transforming data in data pipeline itself so its faster to load data from a smaller size.
# using lambda for transformation, x is being transformed as its an img in numpy array, y is a label so no transforming that

In [ ]:
scaled = data.as_numpy_iterator()

In [ ]:
batch = scaled.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20)) #using matplotlib to show 4 images 
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img) #cus dataset is between 0 - 1
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
# 5. Split Data

In [ ]:
#splitting dataset bacthes for training dataset, validation dataset, testing dataset
len(data)
train_size = int(len(data)*.7)#70% training data size 
val_size = int(len(data)*.2)  #20% to validation
test_size = int(len(data)*.1) #10% to testing




In [ ]:
train = data.take(train_size) #take 4 batches 
val = data.skip(train_size).take(val_size) #skip first 4 batches then take the next 2
test = data.skip(train_size+val_size).take(test_size) # skip 6


In [ ]:
# Convert labels to one-hot encoded format
train = train.map(lambda x, y: (x, tf.one_hot(y, depth=3)))
val = val.map(lambda x, y: (x, tf.one_hot(y, depth=3)))
test = test.map(lambda x, y: (x, tf.one_hot(y, depth=3)))

In [ ]:
# 6. Build Deep Learning Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import regularizers

In [ ]:
model = Sequential()

In [ ]:
#adding layers sequencially

#The number of filters in the convolutional layers is 32, 64, and 128 respectively. 
#This allows the model to capture more features at different levels of abstraction.

model.add(Conv2D(32, (3,3), 1, activation='relu', input_shape=(256,256,3))) 
model.add(MaxPooling2D()) #condense image 

model.add(Conv2D(64, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())

model.add(Flatten())

#L2 regularization with a weight decay of 0.001 has been added to the fully connected layers.
#This helps prevent overfitting by adding a penalty term to the loss function.

model.add(Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))
#Dropout layers with a rate of 0.5 have been added after each fully connected layer. Dropout randomly drops out a fraction
#of the connections during training which can improve generalization and reduce overfitting.

model.add(Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))# output will give probabilities for each class, used sigmoid for 0 or 1 for two classes

In [ ]:
model.compile('adam', loss=tf.losses.CategoricalCrossentropy(), metrics=['accuracy']) #used binry accuracies 
#adam is an optimizer
model.summary()

In [ ]:
# 7. Training DL Model

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])


In [ ]:
# 8. Ploting Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
# 9. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [ ]:
len(test)

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)


In [ ]:
print(pre.result().numpy(), re.result().numpy(), acc.result().numpy())

In [ ]:
# 10. Test

In [ ]:
#cv2 will display img in BGR
img = cv2.imread('snowstormtest.png')
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))#converting img to show in rgb
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256)) #need to resize
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
#expanding the img to the batch and resizing it and then adding it to the predict method
finalAccuracy = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
finalAccuracy

In [ ]:
class_names = ['Floods', 'Snowstorm', 'Tornado']

predicted_class_index = np.argmax(finalAccuracy)
predicted_class = class_names[predicted_class_index]

print(f'Predicted class is {predicted_class}')

In [ ]:
# 11. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','Classify_tornado_flood_snowstorm.h5'))

In [ ]:
new_model = load_model(os.path.join('models','Classify_tornado_flood_snowstorm.h5'))

In [ ]:
yhatNew = new_model.predict(np.expand_dims(resize/255, 0))

In [ ]:
predicted_class_index_new = np.argmax(yhatNew)
predicted_class_new = class_names[predicted_class_index_new]

print(f'Predicted class is {predicted_class_new}')